In [25]:
# do all necessary imports for this chapter
import matplotlib.pyplot as plt
import numpy as np
from ncon import ncon
from scipy.optimize import minimize
from scipy.sparse.linalg import eigs, LinearOperator, gmres
from scipy.linalg import svd, polar, null_space
from functools import partial
from time import time
from tutorialFunctions import createMPS, normalizeMPS, fixedPoints, rightOrthonormalize, mixedCanonical, expVal2Uniform, expVal2Mixed

In [26]:
def gradCenterTerms(hTilde, A, l=None, r=None):
    """
    Calculate the value of the center terms.
    
        Parameters
        ----------
        hTilde : np.array (d, d, d, d)
            reduced Hamiltonian,
            ordered topLeft-topRight-bottomLeft-bottomRight.
        A : np.array (D, d, D)
            normalized MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array (D, D), optional
            left fixed point of transfermatrix,
            normalized.
        r : np.array (D, D), optional
            right fixed point of transfermatrix,
            normalized.
        
        Returns
        -------
        term1 : np.array (D, d, D)
            first term of gradient,
            ordered left-mid-right.
        term2 : np.array (D, d, D)
            second term of gradient,
            ordered left-mid-right.
    """
    
    # calculate fixed points if not supplied
    if l is None or r is None:
        l, r = fixedPoints(A)
        
    # calculate first contraction
    term1 = ncon((l, r, A, A, np.conj(A), hTilde), ([-1, 1], [5, 7], [1, 3, 2], [2, 4, 5], [-3, 6, 7], [3, 4, -2, 6]))
    
    # calculate second contraction
    term2 = ncon((l, r, A, A, np.conj(A), hTilde), ([6, 1], [5, -3], [1, 3, 2], [2, 4, 5], [6, 7, -1], [3, 4, 7, -2]))
    
    return term1, term2

In [27]:
def reducedHamUniform(h, A, l=None, r=None):
    """
    Regularize Hamiltonian such that its expectation value is 0.
    
        Parameters
        ----------
        h : np.array (d, d, d, d)
            Hamiltonian that needs to be reduced,
            ordered topLeft-topRight-bottomLeft-bottomRight.
        A : np.array (D, d, D)
            normalized MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array (D, D), optional
            left fixed point of transfermatrix,
            normalized.
        r : np.array (D, D), optional
            right fixed point of transfermatrix,
            normalized.
            
        Returns
        -------
        hTilde : np.array (d, d, d, d)
            reduced Hamiltonian,
            ordered topLeft-topRight-bottomLeft-bottomRight.
    """
    
    d = A.shape[1]
    
    # calculate fixed points if not supplied
    if l is None or r is None:
        l, r = fixedPoints(A)
    
    # calculate expectation value
    e = np.real(expVal2Uniform(h, A, l, r))
    
    # substract from hamiltonian
    hTilde = h - e * ncon((np.eye(d), np.eye(d)), ([-1, -3], [-2, -4]))
    
    return hTilde

In [28]:
def EtildeRight(A, l, r, v):
    """
    Implement the action of (1 - Etilde) on a right vector v.
    
        Parameters
        ----------
        A : np.array (D, d, D)
            normalized MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array (D, D), optional
            left fixed point of transfermatrix,
            normalized.
        r : np.array (D, D), optional
            right fixed point of transfermatrix,
            normalized.
        v : np.array (D**2)
            right matrix of size (D, D) on which
            (1 - Etilde) acts,
            given as a vector of size (D**2,)
        
        Returns
        -------
        vNew : np.array (D**2)
            result of action of (1 - Etilde)
            on a right matrix,
            given as a vector of size (D**2,)
    """
    
    D = A.shape[0]
    
    # reshape to matrix
    v = v.reshape(D, D)
        
    # transfermatrix contribution
    transfer = ncon((A, np.conj(A), v), ([-1, 2, 1], [-2, 2, 3], [1, 3]))

    # fixed point contribution
    fixed = np.trace(l @ v) * r

    # sum these with the contribution of the identity
    vNew = v - transfer + fixed

    return vNew.reshape((D ** 2))

In [29]:
def RhUniform(hTilde, A, l=None, r=None):
    """
    Find the partial contraction for Rh.
    
        Parameters
        ----------
        hTilde : np.array (d, d, d, d)
            reduced Hamiltonian,
            ordered topLeft-topRight-bottomLeft-bottomRight,
            renormalized.
        A : np.array (D, d, D)
            normalized MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array (D, D), optional
            left fixed point of transfermatrix,
            normalized.
        r : np.array (D, D), optional
            right fixed point of transfermatrix,
            normalized.
        
        Returns
        -------
        Rh : np.array (D, D)
            result of contraction,
            ordered top-bottom.
    """
    
    D = A.shape[0]
    
    # if l, r not specified, find fixed points
    if l is None or r is None:
        l, r = fixedPoints(A)
    
    # construct b, which is the matrix to the right of (1 - E)^P in the figure above
    b = ncon((r, A, A, np.conj(A), np.conj(A), hTilde), ([4, 5], [-1, 2, 1], [1, 3, 4], [-2, 8, 7], [7, 6, 5], [2, 3, 8, 6]))
    
    # solve Ax = b for x
    A = LinearOperator((D ** 2, D ** 2), matvec=partial(EtildeRight, A, l, r))
    Rh = gmres(A, b.reshape(D ** 2))[0]
    
    return Rh.reshape((D, D))

In [30]:
def gradLeftTerms(hTilde, A, l=None, r=None):
    """
    Calculate the value of the left terms.
    
        Parameters
        ----------
        hTilde : np.array (d, d, d, d)
            reduced Hamiltonian,
            ordered topLeft-topRight-bottomLeft-bottomRight,
            renormalized.
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array (D, D), optional
            left fixed point of transfermatrix,
            normalized.
        r : np.array (D, D), optional
            right fixed point of transfermatrix,
            normalized.
        
        Returns
        -------
        leftTerms : np.array (D, d, D)
            left terms of gradient,
            ordered left-mid-right.
    """
    
    # if l, r not specified, find fixed points
    if l is None or r is None:
        l, r = fixedPoints(A)
    
    # calculate partial contraction
    Rh = RhUniform(hTilde, A, l, r)
    
    # calculate full contraction
    leftTerms = ncon((Rh, A, l), ([1, -3], [2, -2, 1], [-1, 2]))
    
    return leftTerms

In [31]:
def EtildeLeft(A, l, r, v):
    """
    Implement the action of (1 - Etilde) on a left vector v.
    
        Parameters
        ----------
        A : np.array (D, d, D)
            normalized MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array (D, D), optional
            left fixed point of transfermatrix,
            normalized.
        r : np.array (D, D), optional
            right fixed point of transfermatrix,
            normalized.
        v : np.array (D**2)
            left matrix of size (D, D) on which
            (1 - Etilde) acts,
            given as a vector of size (D**2,)
        
        Returns
        -------
        vNew : np.array (D**2)
            result of action of (1 - Etilde)
            on a left matrix,
            given as a vector of size (D**2,)
    """
    
    D = A.shape[0]
    
    # reshape to matrix
    v = v.reshape(D, D)

    # transfer matrix contribution
    transfer = ncon((v, A, np.conj(A)), ([3, 1], [1, 2, -2], [3, 2, -1]))

    # fixed point contribution
    fixed = np.trace(v @ r) * l

    # sum these with the contribution of the identity
    vNew = v - transfer + fixed

    return vNew.reshape((D ** 2))

In [32]:
def LhUniform(hTilde, A, l=None, r=None):
    """
    Find the partial contraction for Lh.
    
        Parameters
        ----------
        hTilde : np.array (d, d, d, d)
            reduced Hamiltonian,
            ordered topLeft-topRight-bottomLeft-bottomRight,
            renormalized.
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array (D, D), optional
            left fixed point of transfermatrix,
            normalized.
        r : np.array (D, D), optional
            right fixed point of transfermatrix,
            normalized.
        
        Returns
        -------
        Lh : np.array (D, D)
            result of contraction,
            ordered bottom-top.
    """
    
    D = A.shape[0]
    
    # if l, r not specified, find fixed points
    if l is None or r is None:
        l, r = fixedPoints(A)
    
    # construct b, which is the matrix to the right of (1 - E)^P in the figure above
    b = ncon((l, A, A, np.conj(A), np.conj(A), hTilde), ([5, 1], [1, 3, 2], [2, 4, -2], [5, 6, 7], [7, 8, -1], [3, 4, 6, 8]))    
    
    # solve Ax = b for x
    A = LinearOperator((D ** 2, D ** 2), matvec=partial(EtildeLeft, A, l, r)) 
    Lh = gmres(A, b.reshape(D ** 2))[0]
    
    return Lh.reshape((D, D))

In [33]:
def gradRightTerms(hTilde, A, l=None, r=None):
    """
    Calculate the value of the right terms.
    
        Parameters
        ----------
        hTilde : np.array (d, d, d, d)
            reduced Hamiltonian,
            ordered topLeft-topRight-bottomLeft-bottomRight,
            renormalized.
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array (D, D), optional
            left fixed point of transfermatrix,
            normalized.
        r : np.array (D, D), optional
            right fixed point of transfermatrix,
            normalized.
        
        Returns
        -------
        rightTerms : np.array (D, d, D)
            right terms of gradient,
            ordered left-mid-right.
    """
    
    # if l, r not specified, find fixed points
    if l is None or r is None:
        l, r = fixedPoints(A)
    
    # calculate partial contraction
    Lh = LhUniform(hTilde, A, l, r)
    
    # calculate full contraction
    rightTerms = ncon((Lh, A, r), ([-1, 1], [1, -2, 2], [2, -3]))
    
    return rightTerms

In [34]:
def gradient(h, A, l=None, r=None):
    """
    Calculate the gradient of the expectation value of h @ MPS A.
    
        Parameters
        ----------
        h : np.array (d, d, d, d)
            Hamiltonian,
            ordered topLeft-topRight-bottomLeft-bottomRight,
            renormalized.
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array (D, D), optional
            left fixed point of transfermatrix,
            normalized.
        r : np.array (D, D), optional
            right fixed point of transfermatrix,
            normalized.
        
        Returns
        -------
        grad : np.array (D, d, D)
            Gradient,
            ordered left-mid-right.
    """
    
    # if l, r not specified, find fixed points
    if l is None or r is None:
        l, r = fixedPoints(A)
        
    # renormalize Hamiltonian
    hTilde = reducedHamUniform(h, A, l, r)
        
    # find terms
    centerTerm1, centerTerm2 = gradCenterTerms(hTilde, A, l, r)
    leftTerms = gradLeftTerms(hTilde, A, l, r)
    rightTerms = gradRightTerms(hTilde, A, l, r)
    
    grad = 2 * (centerTerm1 + centerTerm2 + leftTerms + rightTerms)
    
    return grad

In [35]:
def groundStateGradDescent(h, D, eps=1e-1, A0=None, tol=1e-4, maxIter=1e4, verbose=True):
    """
    Find the ground state using gradient descent.
    
        Parameters
        ----------
        h : np.array (d, d, d, d)
            Hamiltonian to minimize,
            ordered topLeft-topRight-bottomLeft-bottomRight.
        D : int
            Bond dimension
        eps : float
            Stepsize.
        A0 : np.array (D, d, D)
            normalized MPS tensor with 3 legs,
            ordered left-bottom-right,
            initial guess.
        tol : float
            Tolerance for convergence criterium.
        
        Returns
        -------
        E : float
            expectation value @ minimum
        A : np.array (D, d, D)
            ground state MPS,
            ordered left-mid-right.
    """
    
    d = h.shape[0]
    
    # if no initial value, choose random
    if A0 is None:
        A0 = createMPS(D, d)
        A0 = normalizeMPS(A0)
    
    # calculate gradient
    g = gradient(h, A0)
    
    A = A0
    
    i = 0
    while not(np.linalg.norm(g) < tol):
        # do a step
        A = A - eps * g
        A = normalizeMPS(A)
        i += 1
        
        if verbose and not(i % 100):
            E = np.real(expVal2Uniform(h, A))
            print('iteration:\t{:d}\tenergy:\t{:.12f}\tgradient norm:\t{:.4e}'.format(i, E, np.linalg.norm(g)))
        
        # calculate new gradient
        g = gradient(h, A)
        
        if i > maxIter:
            print('Warning: gradient descent did not converge!')
            break
    
    # calculate ground state energy
    E = np.real(expVal2Uniform(h, A))
    
    return E, A

In [36]:
def Heisenberg(Jx, Jy, Jz, hz):
    """
    Construct the spin-1 Heisenberg Hamiltonian for given couplings.
    
        Parameters
        ----------
        Jx : float
            Coupling strength in x direction
        Jy : float
            Coupling strength in y direction
        Jy : float
            Coupling strength in z direction
        hz : float
            Coupling for Sz terms

        Returns
        -------
        h : np.array (3, 3, 3, 3)
            Spin-1 Heisenberg Hamiltonian.
    """
    Sx = np.array([[0, 1, 0], [1, 0, 1], [0, 1, 0]]) / np.sqrt(2)
    Sy = np.array([[0, -1, 0], [1, 0, -1], [0, 1, 0]]) * 1.0j /np.sqrt(2)
    Sz = np.array([[1, 0, 0], [0, 0, 0], [0, 0, -1]])
    I = np.eye(3)

    return -Jx*ncon((Sx, Sx), ([-1, -3], [-2, -4]))-Jy*ncon((Sy, Sy), ([-1, -3], [-2, -4]))-Jz*ncon((Sz, Sz), ([-1, -3], [-2, -4])) \
            - hz/2*ncon((I, Sz), ([-1, -3], [-2, -4])) - hz/2*ncon((Sz, I), ([-1, -3], [-2, -4]))

In [37]:
d, D = 3, 12
A = createMPS(D, d)
A = normalizeMPS(A)

h = Heisenberg(-1, -1, -1, 0)

# energy optimization using naive gradient descent
# for D=12 or higher: tolerance lower than 1e-2 gives very long runtimes
print('Gradient descent optimization:\n')
t0 = time()
E1, A1 = groundStateGradDescent(h, D, eps=1e-1, A0=A, tol=1e-2, maxIter=1e4)
print('Time until convergence:', time()-t0, 's')
print('Computed energy:', E1, '\n')

Gradient descent optimization:

iteration:	100	energy:	-1.391769957092	gradient norm:	4.2302e-02
iteration:	200	energy:	-1.396822479267	gradient norm:	1.6067e-02
iteration:	300	energy:	-1.398546292997	gradient norm:	1.0418e-02
Time until convergence: 10.453903436660767 s
Computed energy: -1.398629469952776 

